# Generale

<b>ELABORATO SCRITTO</b>: Salva le varie prove di preprocessing e poi dividi i vari formati nelle immagini di test trovati e come reagiscono a diversi valori di soglia, luminosita' e contrasto per fare un confronto. Commenta l'utilizzo delle due tecniche dopo la detection e delle varie configurazioni possibili per il detector, quale sia meglio usare e quale no. In generale confronta tutti procedimenti fatti, nulla e' stato inventato o fatto da zero ma sono state fatte varie prove. Fai le varie prove con vari preprocessing e poi esegui tutta la pipeline per ogni cartella preprocessata per vedere come finisce tutto e poi confronta

<b>Controllato e i file trainati di tesseract funzionano se non si prova su google colab quindi la parte di riconoscimento con quelli la faccio dopo sul computer confrontando tutti e 3 i modelli (ita, dot1 e dot2)</b>

## Installazione tesseract, altro necessario e mount di GDrive

In [ ]:
!sudo apt install tesseract-ocr
!sudo apt install tesseract-ocr-ita
!pip install pytesseract
!sudo apt install git
from google.colab import drive
drive.mount('/content/drive')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,833 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

## Conversione immagini da bmp a jpg
(CRAFT, ad esempio, non accetta bmp)

In [ ]:
import cv2
from os import listdir, mkdir
from os.path import isfile, join, isdir, exists
import numpy as np
from scipy.ndimage import zoom

def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        #out = np.zeros_like(img)
        # White-padding
        out=np.full_like(img,255)

        #out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)
        out[top:top+zh, left:left+zw]=cv2.resize(img, None, fx=zoom_factor, fy=zoom_factor)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

rootFolder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto'
dirs=['buoni','scarti']
for dir in dirs:
  if (not exists(join(rootFolder,dir+"_jpg"))):
    mkdir(join(rootFolder,dir+"_jpg"))
  files=[f for f in listdir(join(rootFolder,dir)) if isfile(join(rootFolder,dir,f))]
  for file in files:
    img=cv2.imread(join(rootFolder,dir,file))
    new=clipped_zoom(img,0.9)
    #new=cv.resize(zoomed, (1280, 1280))
    cv2.imwrite(join(rootFolder,dir+"_jpg",file.split('.')[0]+".jpg"),new)

## Pre-processing

Basta trovare una soglia che in generale possa far risaltare il testo per la fase di detection. Dopo la fase di detection il riquadro viene ritagliato dall'immagine originale e poi si riapplica un processing piu' leggero visto che a quel punto le zone individuate dovrebbero contenere solo testo su uno sfondo che possiamo eliminare facilmente con qualche soglia magari. Piu' aggressivo ora per eliminare eventuali problemi e rendere molto affidabile la fase di detection, poi qualcosa di meno distruttivo verso i contorni delle lettere magari per utilizzare al meglio tesseract
<br/><br/>

<b>DONE/TODO</b>: Prova ad aumentare il contrasto a 30 e anche la luminosita' (valore di GIMP)<br/>
<b>IDEA</b>: Implementazione ad-hoc di un algoritmo per la soglia dipendentemente dall'immagine finale che si avra' dall'azienda che si occupa della fase di acquisizione
<br/><br/>
<b>Giocare un po' con i valori di luminosita', contrasto e threshold per trovare un buon compromesso per la detection. Non sembra possibili agire solo su luminosita' e contrasto per avere meno errori possibili durante la fase di detection ma bisogna passare al thresholding. (E' anche vero che non abbiamo ancora immagini aggiornate quindi in futuro potrebbe migliorare il tutto) Se ci fossero troppi errori sarebbe da implementare la parte di codice che rileva il punto a concentrazione maggiore di bounding box se si utilizzasse il metodo 1</b>

<b>DONE/TODO</b>: Problemi con l'immagine 2 e 3, provare ad applicare una "ricostruzione" allargando i punti che a volte sono troppo lontani! Usata una soglia diversa per queste due immagini, quando in produzione avremo un formato standard dell'immagine acuquisita avremo una soglia unica ma per ora usiamo qualcosa ad hoc per queste 2 che altrimenti non funzionano

<b>Non funziona il resize della funzione ma non sembra creare problemi. Nei consigli su come usare tesseract e preparare le immagini pero' consigliano una risoluzione di 300DPI che ora non riesco a fare. Controllando i risultati vedo se sistemare</b>

Scoperto che CRAFT funziona molto bene se si inverte l'immagine, il testo diventa bianco e quindi sembra ci siano meno anomalie rilevate come testo!

Notare che per il pre processing delle immagini nel formato 1, 2 e 4 basta una soglia accuratamente scelta (meglio comunque metter mano anche a contrasto e luminosita' per evitare di prendere troppo poco)

<b>
IMAGE_SIZE = 1280
-----IDEA: Implementazione ad-hoc di un algoritmo per la soglia dipendentemente dall'immagine finale che si avra' dall'azienda che si occupa della fase di acquisizione
BINARY_THREHOLD = 50
-----Heavy pre-process se usiamo le non invertite, si potrebbe sperimentare ancora con i valori ma fino a che non si ha un formato standard non troveremo mai molto...per adesso se ne usa solo uno visto che poi avremo un solo tipo di immagini
BRIGHTNESS=-20 # preproc=-20/inverted=[20 35 40]
CONTRAST=30 # preproc=30/inverted=[45 25 40]
</b>

In [ ]:
import tempfile
import cv2
import numpy as np
from PIL import Image
from os import listdir, mkdir
from os.path import isfile, join, exists


# Da questo post di stackoverflow: https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv
def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

# Da questo post di stackoverflow: https://stackoverflow.com/questions/28935983/preprocessing-image-for-tesseract-ocr-with-opencv

def process_image_for_ocr(file_path, brightness, contrast, threshold, img_size=1280):
    # TODO : Implement using opencv
    temp_filename = set_image_dpi(file_path, img_size)
    im_new = remove_noise_and_smooth(temp_filename, brightness, contrast, threshold)
    return im_new

def set_image_dpi(file_path, img_size):
    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = max(1, int(img_size / length_x))
    size = factor * length_x, factor * width_y
    # size = (1800, 1800)
    im_resized = im.resize(size, Image.ANTIALIAS)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(300, 300))
    return temp_filename

def image_smoothening(img, brightness, contrast, threshold):
    img=apply_brightness_contrast(img,brightness,contrast)
    ret1, th1 = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    ret2, th2 = cv2.threshold(th1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    blur = cv2.GaussianBlur(th2, (1, 1), 0)
    ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return th3

def remove_noise_and_smooth(file_name, brightness, contrast, threshold):
    img = cv2.imread(file_name, 0)
    filtered = cv2.adaptiveThreshold(img.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 41,3)
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    img = image_smoothening(img, brightness, contrast, threshold)
    or_image = cv2.bitwise_or(img, closing)
    return or_image

# Funzione per incicciottire il font e renderlo "continuo" da puntinato
# Vuole in input l'immagine in bianco su nero e non nero su bianco
# Restituisce poi un'immagine in bianco e nero

def incicciottisci(img, kernelsize=3, shape=cv2.MORPH_ELLIPSE):
    kernel = cv2.getStructuringElement(shape,(kernelsize,kernelsize))
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    dilation = cv2.dilate(opening,kernel,iterations = 2)
    return (255-dilation)

images_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto'
preproc_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/preprocessing'
dirs=['buoni_jpg','scarti_jpg']
method=3
if (not exists(preproc_folder)):
    mkdir(preproc_folder)
#formato124={"buoni_jpg":["Img0.jpg","Img1.jpg","Img2.jpg","Img3.jpg","Img8.jpg","Img9.jpg"]}
#formato3={"buoni_jpg":["Img4.jpg","Img5.jpg","Img6.jpg","Img7.jpg"]}
for dir in dirs:
    saveto=join(preproc_folder,dir+'_preproc_'+"{}".format(method))
    if (not exists(saveto)):
        mkdir(saveto)
    files=[f for f in listdir(join(images_folder,dir)) if isfile(join(images_folder,dir,f))]
    for file in files:
        if method==0:
            img=cv2.imread(join(images_folder,dir,file), 0)
            final=apply_brightness_contrast(img,40,40)
        elif method==1:
            # Pre processing con la funzione trovata online
            final=process_image_for_ocr(join(images_folder,dir,file),-20,30,50)
            final=incicciottisci(255-final)
        elif method==2:
            img=cv2.imread(join(images_folder,dir,file), 0)
            bc=apply_brightness_contrast((255-img),40,40)
            blur = cv2.blur(bc,(3,3))
            _, thr = cv2.threshold(blur, 250, 255, cv2.THRESH_BINARY_INV)
            # 3 o 5 come kernel size
            final=incicciottisci(255-thr)
        elif method==3:
            # Tarato sull'immagine circa peggiore trovata, la 27 degli scarti, da tarare poi in base al formato finale delle immagini catturate
            img=cv2.imread(join(images_folder,dir,file), 0)
            bc=apply_brightness_contrast(img,50,50)
            blur= cv2.blur(bc,(5,5))
            final=incicciottisci(255-blur,5)
        cv2.imwrite(join(saveto,file),final)

# CRAFT-pytorch

## Clonazione della repository, download dei modelli e installazione dei moduli richiesti

In [ ]:
from os.path import join, exists
from os import mkdir
#destination_folder='/content/drive/MyDrive/tesi/CRAFT-pytorch'
destination_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/CRAFT-pytorch'
if not exists(destination_folder):
  !git clone https://github.com/clovaai/CRAFT-pytorch '$destination_folder'
if not exists(join(destination_folder,'weights')):
  mkdir(join(destination_folder,'weights'))
if not exists(join(destination_folder,'weights','craft_mlt_25k.pth')):
  filename=join(destination_folder,'weights','craft_mlt_25k.pth')
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ' -O '$filename'
if not exists(join(destination_folder,'weights','craft_refiner_CTW1500.pth')):
  filename=join(destination_folder,'weights','craft_refiner_CTW1500.pth')
  !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XSaFwBkOaFOdtk4Ane3DFyJGPRw6v5bO' -O '$filename'
!pip install torch torchvision opencv-python scikit-image scipy

--2021-06-21 07:57:07--  https://docs.google.com/uc?export=download&id=1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ
Resolving docs.google.com (docs.google.com)... 142.251.33.206, 2607:f8b0:4004:82a::200e
Connecting to docs.google.com (docs.google.com)|142.251.33.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-9g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nkur1pa91c776gqli9bc0eoba26t6495/1624262175000/05173585031594261940/*/1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ?e=download [following]
--2021-06-21 07:57:09--  https://doc-0k-9g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nkur1pa91c776gqli9bc0eoba26t6495/1624262175000/05173585031594261940/*/1Jk4eGD7crsqCCg9C9VjCLkMN3ze8kutZ?e=download
Resolving doc-0k-9g-docs.googleusercontent.com (doc-0k-9g-docs.googleusercontent.com)... 172.217.2.97, 2607:f8b0:4004:80a::2001
Connecting to doc-0k-9g-docs.googleusercontent.com (doc-0k-9g-docs.googleu

## Esecuzione
Opzioni di configurazione per CRAFT:
<ul>
  <li><b>--trained_model</b>: pretrained model (default: weights/craft_mlt_25k.pth)</li>
  <li><b>--text_threshold</b>: certainty required for something to be classified as a letter. The higher this value the clearer characters need to look. (recommended 0.5-0.6) (default: 0.7)</li>
  <li><b>--low_text</b>: amount of boundary space around the letter-word when the coordinates are returned. The higher this value the less space. Upping this value also affects the link threshold of seeing words as one, but it can cut off unnecessary borders around letters. Having this value too high can affect edges of letters, cutting them off and lowering the accuracy in reading them. (recommended 0.3-0.4) (default: 0.4)</li>
  <li><b>--link_threshold</b>: amount of distance allowed between two characters for them to be seen as a single word. (recommended 0.1-0.5, however playing with this value for your own use case might be better) (default: 0.4)</li>
  <li><b>--cuda</b>: use cuda for inference (default: True)</li>
  <li><b>--canvas_size</b>: image size for inference (images are resized to this size) (default: 1280)</li>
  <li><b>--mag_ratio</b>: image magnification ratio (default: 1.5)</li>
  <li><b>--poly</b>: enable polygon type (default: False)</li>
  <li><b>--show_time</b>: show processing time (default: False)</li>
  <li><b>--test_folder</b>: folder path containing the images (default: /data/)</li>
  <li><b>--refine</b>: enable link refiner (default: False)</li>
  <li><b>--refine_model</b>: pretrained refiner model (default: weights/craft_refiner_CTW1500.pth)</li>
</ul>

<br/>
<b>Notiamo che passando l'immagine preprocessata in bianco e nero ci mette davvero poco a rilevare il tutto, il che potrebbe permetterci di puntare piu' sulla parte di preprocessing anche se prendesse un po' piu' tempo</b> (interessante scrivere il confronto di tempi sull'elaborato di tesi magari))
<br/><br/>
<b>Tratta il caso sia con che senza il Refiner, usandolo sarebbe meglio ricorrere al metodo 1 (da rivedere ora che nel preprocessing uso anche la funzione per incicciottire i punti). Usando il refiner sembra sia molto meglio il risultato ma alcune bounding box sono sovrapposte e dovrebbe essere fatto il merge in una unica per evitare di riconoscere 2 volte la stessa parole o riconoscerne un pezzo e non essere sicuri poi del risultato...</b>

In [ ]:
#CRAFT_folder='/content/drive/MyDrive/tesi/CRAFT-pytorch'
CRAFT_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/CRAFT-pytorch'
%cd '$CRAFT_folder'
from os import rename
from os.path import join, exists
from os import mkdir
test_path=join(CRAFT_folder,'test.py')
results_path=join(CRAFT_folder,'result')
new_results_path=['buoni_results_refiner_{}','buoni_results_{}','scarti_results_refiner_{}','scarti_results_{}']
preproc_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/preprocessing'
method=3
buoni_dir=join(preproc_folder,"buoni_jpg_preproc_{}".format(method))
scarti_dir=join(preproc_folder,"scarti_jpg_preproc_{}".format(method))
detect=False
if detect:
  !python '$test_path' --test_folder='$buoni_dir'
  rename(results_path, join(CRAFT_folder,new_results_path[1].format(method)))
  !python '$test_path' --test_folder='$buoni_dir' --refine
  rename(results_path, join(CRAFT_folder,new_results_path[0].format(method)))
  !python '$test_path' --test_folder='$scarti_dir'
  rename(results_path, join(CRAFT_folder,new_results_path[3].format(method)))
  !python '$test_path' --test_folder='$scarti_dir' --refine
  rename(results_path, join(CRAFT_folder,new_results_path[2].format(method)))
    

toshared=False
results_copy_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/CRAFT_results'
if (not exists(results_copy_folder)):
    mkdir(results_copy_folder)

if toshared:
  for path in new_results_path:
    path=path.format(method)
    temp=join(CRAFT_folder,path)
    !cp -R '$path' '$results_copy_folder'

/content/drive/MyDrive/tesi/CRAFT-pytorch


## Metodo 1 (ABBANDONATO E NON AGGIORNATO, I PERCORSI NON SONO CORRETTI...)

Singolo rettangolo che contiene tutto il testo delle bounding box ma richiede che il nostro pre-processing porti alla detection del solo testo corretto e all'eliminazione di qualsiasi altra anomalia

Si potrebbe cercare di prendere solo le aree "dense di bounding boxes" e non considerare le altre

Lo skew viene calcolato facendo una media dell'angolo di ogni bounding box quindi risulta facile ad errori se una bounding box viene trovata un po' a caso (in alcuni casi viene trovata una bounding box che contiene le sole coppie di ":" e non sempre ha un'angolazione corretta, portanto ad errore nella media poi)

Aggiustamento possibilie: Controlla prima se esiste una zona con una concentrazione di punti maggiore altrimenti usa il metodo 2 poi, magari ha un suo senso come soluzione

Il metodo 1 ha sicuramente molti problemi negli scarti dove le bounding box potrebbero essere molte e neanche vicine causando inutilita' nelle considerazioni fatte...probabilmente e' meglio il metodo 2 che analizza ogni singola boundibg box

Il modo migliore sarebbe: cercare l'area con una concentrazione di punti maggiore cosi' da elimintare possibili outliers (dato che siamo abbastanza certi di trovare una zona con molte bounding box, almeno 4, e altre sparse in giro se il preprocessing viene fatto bene), utilizzare un qualche metodo per trovare il rettangolo piu' grande che possa contenere tutte le bounding box e poi leggere una singola bounding box. Per ora prendo solo la x minore, maggiore e la y minore e maggiore per poi creare un riquadro con queste dimensioni. Questo caso potrebbe non funzionare nel momento in cui le immagini con anomalie deformano il testo e quindi leggere le singole bounding box risulta meglio che crearne una sola, inoltre il processo potrebbe essere davvero troppo oneroso e la possibilita' di errori (vista la mancanza di immagini con un formato standard in questo momento) viene ad essere alta. Propenderei per il metodo 2 che semplifica il tutto...

### Pre-processing Tesseract NEW

<b>Metodo che funziona solo se viene implementata la funzione che trova il punto a maggiore concentrazione di rettangoli. Vedendo l'analisi degli scarti notiamo come i problemi di illuminazione portano al rilevamento di molte bounding box presumibilmente contententi testo e questo andrebbe a rendere totalmente inutile anche questa modifica in questa fase. Inoltre l'algoritmo per la zona maggiormente densa di bounding box, che si riduce ad un calcolo del centro della regione piu' densa di punti, dovrebbe lavorare dinamicamente e valutare anche il caso in cui ci sia il rettangolo piu' grande ma sempre guardando i casi di scarto questo metodo non e' infallibile. Se in futuro potesse migliorare l'illuminazione, potendo in questo modo tarare un pre processing piu' fine, potremmo ricorrere a questa tecnica che sicuramente ci aiuta molto. Utilizzando questa potremmo anche usare tesseract per verifica se il testo e' capovolto oppure no, cosa che adesso non possiamo fare dato che avendo troppi pochi caratteri tesseract ci restituisce errore... Riportiamo il primo metodo solo per i casi buoni, nel caso degli scarti produce troppi errori, basta guardare ai casi delle immagini con tanti riquadri trovati. Si potrebbe pensare ad una soluzione che in base al numero di rettangolo rilevati utilizzi il metodo 1 (in caso di massimo 10 rettangoli trovati, accenttando ci siano degli outliers, valutando anche il caso di rettangolo piu' grande di altri) e metodo 2 in caso ci siano troppi rettangoli per evitare di commettere errori e analizzare ogni singola area di testo potenzialmente rilevata</b>

image_to_osd di pytesseract ha bisogno di almeno 50 caratteri, potremmo provare a giocare con min_characters_to_try ma non dovrebbe comunque bastare

In [ ]:
import cv2
import numpy as np
import math
from math import sqrt
from os import mkdir, listdir
from os.path import isfile, join, exists
from scipy.ndimage import zoom
from scipy.spatial import ConvexHull
from scipy.spatial import distance as dist

def order_points(pts):
	# sort the points based on their x-coordinates
	xSorted = pts[np.argsort(pts[:, 0]), :]
	# grab the left-most and right-most points from the sorted
	# x-roodinate points
	leftMost = xSorted[:2, :]
	rightMost = xSorted[2:, :]
	# now, sort the left-most coordinates according to their
	# y-coordinates so we can grab the top-left and bottom-left
	# points, respectively
	leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
	(tl, bl) = leftMost
	# now that we have the top-left coordinate, use it as an
	# anchor to calculate the Euclidean distance between the
	# top-left and right-most points; by the Pythagorean
	# theorem, the point with the largest distance will be
	# our bottom-right point
	D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
	(br, tr) = rightMost[np.argsort(D)[::-1], :]
	# return the coordinates in top-left, top-right,
	# bottom-right, and bottom-left order
	return np.array([tl, tr, br, bl], dtype="int32")

def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        #out = np.zeros_like(img)
        # White-padding
        out=np.full_like(img,255)

        #out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)
        out[top:top+zh, left:left+zw]=cv2.resize(img, None, fx=zoom_factor, fy=zoom_factor)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

def rotate_image(image, angle, center):
  # Questa volta ruoto secondo il punto contro cui calcolo lo skew
  image_center = tuple(np.array(center))
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], borderValue=(255,255,255))
  return result

def minimum_bounding_rectangle(points):
    """
    Find the smallest bounding rectangle for a set of points.
    Returns a set of points representing the corners of the bounding box.

    :param points: an nx2 matrix of coordinates
    :rval: an nx2 matrix of coordinates
    """
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.

    # get the convex hull for the points
    hull_points = points[ConvexHull(points).vertices]

    # calculate edge angles
    edges = np.zeros((len(hull_points)-1, 2))
    edges = hull_points[1:] - hull_points[:-1]

    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])

    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)

    # find rotation matrices
    # XXX both work
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles-pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T
#     rotations = np.vstack([
#         np.cos(angles),
#         -np.sin(angles),
#         np.sin(angles),
#         np.cos(angles)]).T
    rotations = rotations.reshape((-1, 2, 2))

    # apply rotations to the hull
    rot_points = np.dot(rotations, hull_points.T)

    # find the bounding points
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)

    # find the box with the best area
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)

    # return the best box
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    r = rotations[best_idx]

    rval = np.zeros((4, 2))
    rval[0] = np.dot([x1, y2], r)
    rval[1] = np.dot([x2, y2], r)
    rval[2] = np.dot([x2, y1], r)
    rval[3] = np.dot([x1, y1], r)

    return rval

work_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_1'
images_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto'
CRAFT_folder='/content/drive/MyDrive/tesi/CRAFT-pytorch'

if not exists(work_folder):
  mkdir(work_folder)
if not exists(join(work_folder,'buoni')):
  mkdir(join(work_folder,'buoni'))
if not exists(join(work_folder,'scarti')):
  mkdir(join(work_folder,'scarti'))
for folder in ['buoni_results_refiner_preproc']: #['buoni_results','scarti_results']:
  files=[f.split('.')[0] for f in listdir(join(CRAFT_folder,folder)) if isfile(join(CRAFT_folder,folder,f)) and (f.split('.')[1]=='txt')]
  for file in files:
    with open(join(CRAFT_folder,folder,file+'.txt')) as fin:
      points=[]
      x=[]
      y=[]
      for line in fin:
          if len(line)!=1:
            coord=line.rstrip().split(',')
            for i in range(0,len(coord),2):
              points.append([int(coord[i]),int(coord[i+1])])
    if points!=None:
      pts=np.array(minimum_bounding_rectangle(np.array(points)),np.int32)
      pts=order_points(pts)
      # Calcolo l'angolazione della bounding box negandola in segno per poi sistemare lo skew (potrebbe non essere il metodo migliore se il detector sbaglia con la bounding box)
      diffs=[pts[0,0]-pts[3,0],pts[0,1]-pts[3,1]]
      angle=-(math.atan(diffs[0]/diffs[1])*180/math.pi)
      # Calcolo la lunghezza dei lati del rettangolo della bounding box
      dist1=math.ceil(sqrt((pts[3,0]-pts[0,0])**2+(pts[3,1]-pts[0,1])**2))
      dist2=math.ceil(sqrt((pts[1,0]-pts[0,0])**2+(pts[1,1]-pts[0,1])**2))
      # Ruoto l'immagine rispetto al punto rispetto a cui ho calcolato l'angolo
      im=cv2.imread(join(images_folder,folder.split('_')[0]+'_jpg_preproc',file.split('_')[1]+'.jpg'),0)
      final=rotate_image(im,angle,[pts[0,0],pts[0,1]])
      # Ritaglio la zona di interesse
      final=final[pts[0,1]:pts[0,1]+dist1,pts[0,0]:pts[0,0]+dist2]
      # Se in verticale la giro
      if final.shape[0]>final.shape[1]:
        final=cv2.rotate(final,0)
      # TODO: Applica il pre-processing prima di fare lo zoom per portare tutto in bianco e nero (dovrebbe essere piu' semplice la questione relativa alle soglie a questo punto)
      
      #final=remove_noise_and_smooth(final)
      #print(final[::].max())
      # Faccio uno zoom di 0.5 per "aggiungere area"
      final=clipped_zoom(final,0.5)
      final_flipped=cv2.rotate(final,1)
      # Salvo
      cv2.imwrite(join(work_folder,folder.split('_')[0],file.split('_')[1]+'.jpg'),final)
      cv2.imwrite(join(work_folder,folder.split('_')[0],file.split('_')[1]+'_flipped.jpg'),final_flipped)
    else:
      print("Non e' stata rilevata la presenza di alcun testo (nessuna boundig box) nell'immagine "+join(images_folder,folder.split('_')[0]+'_jpg',file.split('_')[1]+'.jpg'))

AttributeError: ignored

### Pre-processing Tesseract OLD <b>(NON USARE)</b>
<b>DONE/TODO</b>: Ritaglia dall'immagine originale e non pre-processata e poi riesegui il preprocessing<br/>
<b>DONE/TODO</b>: Sistema in caso non ci siano bounding box rilevate...<br/>
<b>TODO</b>: In base alla grandezza del rettangolo trovato che contiene tutto prova a fare uno zoom variabile (?)<br/>
<b>TODO</b>: Cerca i rettangoli piu' vicini prima di raddrizzare, in questo modo elimini i casi strani oppure lavora sul preprocessing (utilizzalo in congiunzione al metodo sopra per riunire tutto per bene)


In [ ]:
import cv2 as cv
import numpy as np
from os import listdir, mkdir
from os.path import isfile, join, exists
import math
from statistics import mean
from scipy.ndimage import zoom

def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        #out = np.zeros_like(img)
        out=np.full_like(img,255)

        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

def rotate_image(image, angle):
  # Ruotiamo l'immagine rispetto al centro dell'immagine stessa. Il ritaglio poi dovrebbe starci tutto anche nel rettangolo singolo che andiamo a calcolare
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LINEAR,borderValue=(255,255,255))
  return result

def skew_angle(filename):
  # Calcola l'angolo medio di sfasamento fra tutte le bounding box rilevate
  with open(filename) as fin:
    angle=[]
    for line in fin:
      if len(line)!=1:
        coord=line.rstrip().split(',')
        diffs=[int(coord[0])-int(coord[-2]),int(coord[1])-int(coord[-1])]
        angle.append(-(math.atan(diffs[0]/diffs[1])*180/math.pi))
  return mean(angle)
			  

def single_bounding_box(filename):
  # Difficilmente potremo trovare una singola bounding box e anche se la trovassimo perderemmo l'informazione sull'angolazione del testo, dovendo quindi raddrizzare in un altro modo
  # Questo sembra un metodo abbastanza funzionante (genero i punti usando le x e le y minori e maggiori), sicuramente porta a livelli di zoom diversi e quindi dovremo valutare in fase di testing cosa comporta
  with open(filename) as fin:
    x=[]
    y=[]
    rot=[]
    for line in fin:
        if len(line)!=1:
          coord=line.rstrip().split(',')
          x.append(coord[0::2])
          y.append(coord[1::2])
    flat_list_x = [int(item) for sublist in x for item in sublist]
    flat_list_y = [int(item) for sublist in y for item in sublist]
  return [[min(flat_list_x),min(flat_list_y)],[max(flat_list_x),min(flat_list_y)],[max(flat_list_x),max(flat_list_y)],[min(flat_list_x),max(flat_list_y)]]

def link_dots(img):
    #kernel=cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))
    #kernel=cv.getStructuringElement(cv.MORPH_CROSS,(3,3))
    #kernel=cv.getStructuringElement(cv.MORPH_RECT,(3,3))
    matrix=[[3,5,3],[5,9,5],[3,5,3]]
    kernel=np.array(matrix,np.uint8)
    opening = cv.morphologyEx(img, cv.MORPH_OPEN, kernel)
    dilation = cv.dilate(opening,kernel,iterations = 2)
    return (255-dilation)


# TODO: Ritaglia dall'immagine originale e non pre-processata e poi riesegui il preprocessing 
# DONE/TODO: Sistema in caso non ci siano bounding box rilevate...
# TODO: In base alla grandezza del rettangolo trovato che contiene tutto prova a fare uno zoom variabile (?)
# TODO: Cerca i rettangoli piu' vicini prima di raddrizzare, in questo modo elimini i casi strani oppure lavora sul preprocessing

# Notare che il caso dell'immagine 7 non funziona bene visto che la soglia scelta nel pre-processing porta a rilevare anche altro e ad una bounding box sbagliata di conseguenza... Sistemare quello o
# guardare questo link https://stackoverflow.com/questions/57539749/find-out-centre-of-the-most-dense-region-in-a-scatter-plot per trovare il punto piu' denso di punti nella nostra immagine, ovvero
# in cui si concentrano piu' bounding box ed eliminare le altre, quelle lontane dal punto "caldo", che saranno presumibilmente errori

work_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_1'
images_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto'
CRAFT_folder='/content/drive/MyDrive/tesi/CRAFT-pytorch'

if not exists(work_folder):
  mkdir(work_folder)
if not exists(join(work_folder,'buoni')):
  mkdir(join(work_folder,'buoni'))
if not exists(join(work_folder,'scarti')):
  mkdir(join(work_folder,'scarti'))
for folder in ['buoni_results']: #['buoni_results','scarti_results']:
  files=[f.split('.')[0] for f in listdir(join(CRAFT_folder,folder)) if isfile(join(CRAFT_folder,folder,f)) and (f.split('.')[1]=='txt')]
  for file in files:
    filename=join(CRAFT_folder,folder,file+'.txt')
    box=single_bounding_box(filename)
    angle=skew_angle(filename)
    if box!=None and angle!=None:
      im=cv.imread(join(images_folder,folder.split('_')[0]+'_jpg',file.split('_')[1]+'.jpg'))
      # Possibile problema in caso non ci stia la rotazione...guardare https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/ in caso
      final=rotate_image(im[box[0][1]:box[2][1], box[0][0]:box[1][0]],angle)
      # TODO: Applica il pre-processing prima di fare lo zoom per portare tutto in bianco e nero (dovrebbe essere piu' semplice la questione relativa alle soglie a questo punto)
      # TODO: Ingrassare i puntini o usare file di tesseract trainato su font a matrice di punti
      #final=clipped_zoom(link_dots(255-final),0.5)
      final=clipped_zoom(final,0.5)
      if final.shape[0]>final.shape[1]:
        final=cv.rotate(final,0)
      final_flipped=cv.rotate(final,1)
      cv.imwrite(join(work_folder,folder.split('_')[0],file.split('_')[1]+'.jpg'),final)
      cv.imwrite(join(work_folder,folder.split('_')[0],file.split('_')[1]+'_flipped.jpg'),final_flipped)
    else:
      print("Non e' stata rilevata la presenza di alcun testo (nessuna boundig box) nell'immagine "+join(images_folder,folder.split('_')[0]+'_jpg',file.split('_')[1]+'.jpg'))

### Riconoscimento
Provare ad utilizzare il modello trainato di tesseract per i display a matrice di punti trovato su github per vedere se performa meglio dell'iniciccionimento dei punti per cercare di far diventare il font puntinato continuo


In [ ]:
import pytesseract
from os.path import exists, join, isfile
from os import listdir
from shutil import copyfile

traineddata_folder='/content/drive/MyDrive/tesi'
tessdata_folder='/usr/share/tesseract-ocr/4.00/tessdata'
if not exists(join(traineddata_folder,'LCDDot_FT_500.traineddata')):
  !wget https://github.com/ameera3/OCR_Expiration_Date/blob/master/OCR_Ensemble/LCDDot_FT_500.traineddata
if not exists(join(traineddata_folder,'dotslayer.traineddata')):
  !wget https://github.com/Shreeshrii/tessdata_shreetest/blob/master/dotslayer.traineddata

copyfile(join(traineddata_folder,'LCDDot_FT_500.traineddata'), join(tessdata_folder,'LCDDot_FT_500.traineddata'))
copyfile(join(traineddata_folder,'dotslayer.traineddata'), join(tessdata_folder,'dotslayer.traineddata'))
#from os import listdir, chdir, mkdir
#from os.path import isfile, join, isdir, exists
#import cv2 as cv
#root_folder_images='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/cut'
#files=[f for f in listdir(root_folder_images) if isfile(join(root_folder_images,f)) and (f.split('.')[1]=='jpg')]
#for file in files:
#  print(join(root_folder_images,file))
#  print(pytesseract.image_to_string(join(root_folder_images,file)))
#  print('\n')

# Sintassi generale per usare il comando con le opzioni di configurazione per tesseract
#target = pytesseract.image_to_string(image, lang='eng', boxes=False, config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')
# Da prelevare l'immagine giusta o la cartella con le immagini di tutte le boundibg box ritagliate
detected_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_1'
first_method_ita=[]
first_method_LCDDot=[]
first_method_dotslayer=[]
for folder in ['buoni']:
  images=[f for f in listdir(join(detected_folder,folder)) if isfile(join(detected_folder,folder,f))]
  images.sort()
  for image in images:
    img=join(detected_folder,folder,image)
    print(img)
    #print(pytesseract.image_to_string(img,config='--psm 0 -c min_characters_to_try=5'))
    # In caso si utilizzi il metodo 1: nessuna configurazione particolare e andiamo alla buona per riconoscere le 2 righe
    # (notiamoche di default tesseract si aspetta un blocco lungo di testo e magari crea errore)
    first_method_ita.append(pytesseract.image_to_string(img, lang='ita'))
    print(first_method_ita[-1])
    #first_method_LCDDot.append(pytesseract.image_to_string(img, lang='LCDDot_FT_500'))
    #first_method_dotslayer.append(pytesseract.image_to_string(img, lang='dotslayer'))

## Metodo 2 (UNICO DA USARE CONTINUATO AD ESSERE AGGIORNATO, L'ALTRO POTREBBE NON ESSERE COMPATIBILE CON I CAMBI DI CARTELLE ORA)

Ritaglio ogni bounding box, creo una cartella che le contiene tutte e poi le analizzo una ad una. A noi interessa solo trovare una data che ha sempre un dash ("-") che la contraddistingue e un numero di lotto che invece non contiene quel carattere. Delle parole "Lotto", "Scad." e dei due punti ci interessa relativamente a questo punto ma non abbiamo la "visione d'insieme" richiesta da Leandro con un singolo riquadro che contiene tutto

Possibile integrare il metodo uno e il metodo 2 per analizzare con il metodo 2 e salvare un risultato con il metodo 1 per scopi di storico magari

Utilizza il secondo link di github trovato e prova se sia meglio riconoscere una sola parola. Per impostare tesseract a riconoscere una sola parola -psm=8 (mi sembra) o guarda sulla pagina del tutorial di tesseract. Imposta i caratteri che potrebbero comparire ed elimina tutti gli altri possibilie (sempre dalle opzioni di tesseract)

Troppi poche lettere per fari si' che tesseract ci possa rilevare se il testo e' sottosopra... L'unica idea possibile e' cercare la parola lotto (quando il formato della scritta lo contiene) e utilizzare il set di immagini (flipped o no) in base a che tipo di immagine questa fosse. Nelle immagini in cui la scritta lotto non compare ha senso salvare entrambe le copie del valore a meno che non contengano valori strani...

Non si riescono ad utilizzare i traineddata per display a matrice di punti per non si sa quale motivo, scaricando e installando la lingua italiana per tesseract i risultati migliorano abbastanza

<b>Si potrebbe mettere un limite minimo alla grandezza della bounding box, scartando quelle troppo piccole. Non credo si possa fare lo stesso ragionamento al contrario</b>

Le ultime immagini degli scarti, secondo me, sarebbero facilmente riconoscibili se non si usasse la funzione incicciottisci o con un kernel piu' piccolo. Un kernel piu' piccolo rovinerebbe gli altri casi senza un modello trainato... una volta che si avra' un modello trainato non avremo piu' bisogno neanche della funzione incicciottisti!

### Pre-processing Tesseract
<b>DONE/TODO</b>: Applica il pre-processing prima di fare lo zoom per portare tutto in bianco e nero (dovrebbe essere piu' semplice la questione relativa alle soglie a questo punto)

<b>TODO</b>: Provare diversi valori di soglia per il pre-processing (conviene abbassarla)

In [ ]:
import cv2
import numpy as np
import math
from math import sqrt
from os import mkdir, listdir
from os.path import isfile, join, exists
from scipy.ndimage import zoom
from scipy.spatial import ConvexHull
from scipy.spatial import distance as dist
from PIL import Image

# Ordina i punti derivati dal calcolo del MRB: https://www.pyimagesearch.com/2016/03/21/ordering-coordinates-clockwise-with-python-and-opencv/

def order_points(pts):
	# sort the points based on their x-coordinates
	xSorted = pts[np.argsort(pts[:, 0]), :]
	# grab the left-most and right-most points from the sorted
	# x-roodinate points
	leftMost = xSorted[:2, :]
	rightMost = xSorted[2:, :]
	# now, sort the left-most coordinates according to their
	# y-coordinates so we can grab the top-left and bottom-left
	# points, respectively
	leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
	(tl, bl) = leftMost
	# now that we have the top-left coordinate, use it as an
	# anchor to calculate the Euclidean distance between the
	# top-left and right-most points; by the Pythagorean
	# theorem, the point with the largest distance will be
	# our bottom-right point
	D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
	(br, tr) = rightMost[np.argsort(D)[::-1], :]
	# return the coordinates in top-left, top-right,
	# bottom-right, and bottom-left order
	return np.array([tl, tr, br, bl], dtype="int32")

# Calcolo del MRB: https://gis.stackexchange.com/questions/22895/finding-minimum-area-rectangle-for-given-points
 
def minimum_bounding_rectangle(points):
    """
    Find the smallest bounding rectangle for a set of points.
    Returns a set of points representing the corners of the bounding box.

    :param points: an nx2 matrix of coordinates
    :rval: an nx2 matrix of coordinates
    """
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.

    # get the convex hull for the points
    hull_points = points[ConvexHull(points).vertices]

    # calculate edge angles
    edges = np.zeros((len(hull_points)-1, 2))
    edges = hull_points[1:] - hull_points[:-1]

    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])

    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)

    # find rotation matrices
    # XXX both work
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles-pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T
#     rotations = np.vstack([
#         np.cos(angles),
#         -np.sin(angles),
#         np.sin(angles),
#         np.cos(angles)]).T
    rotations = rotations.reshape((-1, 2, 2))

    # apply rotations to the hull
    rot_points = np.dot(rotations, hull_points.T)

    # find the bounding points
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)

    # find the box with the best area
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)

    # return the best box
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    r = rotations[best_idx]

    rval = np.zeros((4, 2))
    rval[0] = np.dot([x1, y2], r)
    rval[1] = np.dot([x2, y2], r)
    rval[2] = np.dot([x2, y1], r)
    rval[3] = np.dot([x1, y1], r)

    return rval

# Da questo post di stackoverflow: https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv

def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

# Funzione per incicciottire il font e renderlo "continuo" da puntinato
# Vuole in input l'immagine in bianco su nero e non nero su bianco
# Restituisce poi un'immagine in bianco e nero

def incicciottisci(img, kernelsize=3, shape=cv2.MORPH_ELLIPSE):
    kernel = cv2.getStructuringElement(shape,(kernelsize,kernelsize))
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    dilation = cv2.dilate(opening,kernel,iterations = 2)
    return (255-dilation)

# Zoom

def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        #out = np.zeros_like(img)
        # White-padding
        out=np.full_like(img,255)

        #out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)
        out[top:top+zh, left:left+zw]=cv2.resize(img, None, fx=zoom_factor, fy=zoom_factor)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

# Ruotare l'immagine per raddrizzarla

def rotate_image(image, angle, center):
  # Questa volta ruoto secondo il punto contro cui calcolo lo skew
  image_center = tuple(np.array(center))
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], borderValue=(255,255,255))
  return result

# Da: https://stackoverflow.com/questions/4993082/how-can-i-sharpen-an-image-in-opencv

def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

work_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_2'
#work_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/ritagli' # In caso si volessero salvare i ritagli
images_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto'
CRAFT_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/CRAFT_results'
method=3

min_area=None
max_area=None
min_size=None
max_size=None
min_area_img=None
max_area_img=None

if not exists(work_folder):
  mkdir(work_folder)
buoni=join(work_folder,'buoni_{}'.format(method))
if not exists(buoni):
  mkdir(buoni)
buoni_refiner=join(work_folder,'buoni_refiner_{}'.format(method))
if not exists(buoni_refiner):
  mkdir(buoni_refiner)
scarti=join(work_folder,'scarti_{}'.format(method))
if not exists(scarti):
  mkdir(scarti)
scarti_refiner=join(work_folder,'scarti_refiner_{}'.format(method))
if not exists(scarti_refiner):
  mkdir(scarti_refiner)
for folder in ['buoni_results_refiner_{}'.format(method),'scarti_results_refiner_{}'.format(method)]:#,'buoni_results_{}'.format(method),'scarti_results_{}'.format(method)]:
  if exists(join(CRAFT_folder,folder)):
    files=[f.split('.')[0] for f in listdir(join(CRAFT_folder,folder)) if isfile(join(CRAFT_folder,folder,f)) and (f.split('.')[1]=='txt')]
    for file in files:
      name=file[4:]
      if folder.split('_')[2]=='refiner':
        saveto=join(work_folder,folder.split('_')[0]+'_refiner_{}'.format(method),name)
      else:
        saveto=join(work_folder,folder.split('_')[0]+'_{}'.format(method),name)
      if not exists(saveto):
        mkdir(saveto)
      with open(join(CRAFT_folder,folder,file+'.txt')) as fin:
        i=0
        for line in fin:
          # Ritaglio ogni bounding box e la analizzo singolarmente
          # Prendo solo le righe non vuote visto che il test.py di craft stampa una riga vuota dopo ogni riga con delle coordinate
          if(len(line)!=1):
            coord=np.array(line.rstrip().split(',')).astype(np.int).reshape(-1,2)
            if len(coord)>4: # Caso con poligono e non rettangolo/i
              print("Poligono, estraggo la bounding box rettangolare che lo contiene ({})".format(saveto))
              pts=minimum_bounding_rectangle(coord)
              coord=order_points(pts)
            # La boundibg box a volte andava fuori dall'immagine e quindi durante il ritaglio si creavano errori...
            # Dovrebbe non succedere piu' usando una semplice soluzione quale fare un piccolo zoom (0.9) sull'immagine originale durante il preprocessing (anzi, durante la conversione in jpg in questo caso)
            # Calcolo l'angolazione della bounding box negandola in segno per poi sistemare lo skew (potrebbe non essere il metodo migliore se il detector sbaglia l'angolazione della bounding box)
            diffs=[coord[0,0]-coord[-1,0],coord[0,1]-coord[-1,1]]
            angle=-(math.atan(diffs[0]/diffs[1])*180/math.pi)
            # Calcolo la lunghezza dei lati del rettangolo della bounding box
            dist1=math.ceil(sqrt((coord[-1,0]-coord[0,0])**2+(coord[-1,1]-coord[0,1])**2))
            dist2=math.ceil(sqrt((coord[1,0]-coord[0,0])**2+(coord[1,1]-coord[0,1])**2))
            # Ruoto l'immagine rispetto al punto rispetto a cui ho calcolato l'angolo
            # Una volta che si ha un formato standard potremmo prendere l'immagine preprocessata e riconoscere direttamente su questa
            #print(join(images_folder,folder.split('_')[0]+real_directory_postfix,file.split('_')[1]+'.jpg'))
            im=cv2.imread(join(images_folder,folder.split('_')[0]+'_jpg',name+'.jpg'),0)
            rotated=rotate_image(im,angle,[coord[0,0],coord[0,1]])
            # Ritaglio la zona di interesse
            cut=rotated[coord[0,1]:coord[0,1]+dist1,coord[0,0]:coord[0,0]+dist2]
            # Se in verticale la giro
            if cut.shape[0]>cut.shape[1]:
              cut=cv2.rotate(cut,0)

            # Cerco la dimensione minima e massima delle bounding box
            if min_area==None:
              min_area=np.prod(cut.shape[:2])
              min_size=cut.shape[:2]
              min_size_img=join(images_folder,folder.split('_')[0]+'_jpg',name+'.jpg')
            elif min_area>np.prod(cut.shape[:2]):
              min_area=np.prod(cut.shape[:2])
              min_size=cut.shape[:2]
              min_size_img=join(images_folder,folder.split('_')[0]+'_jpg',name+'.jpg')
            if max_area==None:
              max_area=np.prod(cut.shape[:2])
              max_size=cut.shape[:2]
              max_size_img=join(images_folder,folder.split('_')[0]+'_jpg',name+'.jpg')
            elif max_area<np.prod(cut.shape[:2]):
              max_area=np.prod(cut.shape[:2])
              max_size=cut.shape[:2]
              max_size_img=join(images_folder,folder.split('_')[0]+'_jpg',name+'.jpg')
            #######

            if 0==1:  # Per avere i ritagli senza post processing
              final=cut
            elif 0==0:  # Questi 2 post processing sono praticamente equivalenti, il secondo funziona meglio e soprattutto nei casi con strisce luminose che tagliano le lettere ma e' da affinare un attimo (entrambi fatti da me)
              # TODO: Applica il pre-processing prima di fare lo zoom per portare tutto in bianco e nero (dovrebbe essere piu' semplice la questione relativa alle soglie a questo punto)
              #######
              filtered = cv2.adaptiveThreshold(cut.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 41,3)
              bc=apply_brightness_contrast(cut,103,43)
              blur_otsu = cv2.GaussianBlur(bc,(7,7),0)
              _,thr = cv2.threshold(blur_otsu,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
              or_image = cv2.bitwise_or(thr, filtered)
              cic=incicciottisci(255-or_image, 4)  #grande variabilita' con tesseract...4x4 (lo so che sarebbe meglio dispari ma quello che fa al massimo e' traslarmi tutto in diagonale un po' piu' in basso) non riconosce piu' quasi nulla....
              blur= cv2.blur(cic,(3,3))
              _, thr_post_blur = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
              #######
              # Faccio uno zoom di 0.5 per "aggiungere area"
              final=clipped_zoom(thr_post_blur,0.5)
            else:
              # TODO: Applica il pre-processing prima di fare lo zoom per portare tutto in bianco e nero (dovrebbe essere piu' semplice la questione relativa alle soglie a questo punto)
              #######
              filtered = cv2.adaptiveThreshold(cut.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 41,3)
              bc=apply_brightness_contrast(cut,97,67)
              blur_otsu = cv2.GaussianBlur(bc,(9,9),0)
              thr=cv2.adaptiveThreshold(blur_otsu,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,41,5)
              or_image = cv2.bitwise_or(thr, filtered)
              cic=incicciottisci(255-or_image, 4)  #grande variabilita' con tesseract...4x4 (lo so che sarebbe meglio dispari ma quello che fa al massimo e' traslarmi tutto in diagonale un po' piu' in basso) non riconosce piu' quasi nulla....
              blur= cv2.blur(cic,(3,3))
              _, thr_post_blur = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
              #######
              # Faccio uno zoom di 0.5 per "aggiungere area"
              final=clipped_zoom(thr_post_blur,0.5)
            final_flipped=cv2.rotate(final,1)
            # Salvo
            im_pil = Image.fromarray(final)
            im_pil.save(join(saveto,'{}'.format(i)+'.jpg'), dpi=(300.0, 300.0))
            im_pil = Image.fromarray(final_flipped)
            im_pil.save(join(saveto,'{}'.format(i)+'_flipped.jpg'), dpi=(300.0, 300.0))
            #cv2.imwrite(join(saveto,'{}'.format(i)+'.jpg'),final)
            #cv2.imwrite(join(saveto,'{}'.format(i)+'_flipped.jpg'),final_flipped)
            i+=1
        if i==0:
          print("Non e' stata rilevata la presenza di alcun testo (nessuna bounding box) nell'immagine "+join(images_folder,folder.split('_')[0]+'_jpg',name+'.jpg'))
  else:
    print(folder+' non esiste! Eseguire prima la fase di detection...')
print('min_size: ',min_size)
print('min_area: ',min_area)
print('min_size_img: ',min_size_img)
print('max_size: ',max_size)
print('max_area: ',max_area)
print('max_size_img: ',max_size_img)
  

Poligono, estraggo la bounding box rettangolare che lo contiene (/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/ritagli/scarti_refiner_3/Img10)
Poligono, estraggo la bounding box rettangolare che lo contiene (/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/ritagli/scarti_refiner_3/Img14)
Poligono, estraggo la bounding box rettangolare che lo contiene (/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/ritagli/scarti_refiner_3/Img18)
Poligono, estraggo la bounding box rettangolare che lo contiene (/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/ritagli/scarti_refiner_3/Img17)
Poligono, estraggo la bounding box rettangolare che lo contiene (/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/ritagli/scarti_refiner_3/Img11)
Poligono, estraggo la bounding box rettangolare che lo contiene (/content/drive/Shareddrives/Te

### Riconoscimento (da fare su windows o comunque non su colab)
Provare ad utilizzare il modello trainato di tesseract per i display a matrice di punti trovato su github per vedere se performa meglio dell'iniciccionimento dei punti per cercare di far diventare il font puntinato continuo

In [ ]:
# Immagini labellate con il testo che dovremmo riconoscere (basta fare una ricerca sul se la stringa trovata con tesseract e' contenuta nella stringa di ogni chiave del dizionario dell immagini)
# Provare poi sostituendo i caratteri ambigui come "O" e "0"; penso siano gli unici in quanto sostiuire i le "T" con i "7" non sarebbe corretto o i "5" con le "S" (e viceversa tutti i casi)
labels={'Img0': ['LOTTO:L20L0M\nSCAD.:12-2022'],
				'Img1': ['LOTTO:L20L0M\nSCAD.:12-2022'],
				'Img2': ['L20K77\n11-2022'],
				'Img3': ['L20K77\n11-2022'],
				'Img4': ['LOTTO:L20K6X\nSCAD.:11-2023'],
				'Img5': ['LOTTO:L20K6X\nSCAD.:11-2023'],
				'Img6': ['LOTTO:L20K6X\nSCAD.:11-2023'],
				'Img7': ['LOTTO:L20K6X\nSCAD.:11-2023'],
				'Img8': ['L20L09\n12-2022'],
				'Img9': ['L20L09\n12-2022'],
        'Img10': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img11': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img12': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img13': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img14': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img15': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img16': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img17': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img18': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img19': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img20': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img21': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img22': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img23': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img24': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img25': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img26': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img27': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img28': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img29': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img30': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img31': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img32': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img33': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img34': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img35': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img36': [''],
				'Img37': [''],
				'Img38': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img39': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img40': [''],
				'Img40_lowExp': [''],
				'Img41': [''],
				'Img42': [''],
				'Img42_lowExp': [''],
				'Img43': [''],
				'Img44': [''],
				'Img45': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img46': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img47': [''],
				'Img48': [''],
				'Img49': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img50': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img51': ['LOTTO:L20L0L\nSCAD.:12-2022'],
				'Img52': ['LOTTO:L20L0L\nSCAD.:12-2022'],
        }

import pytesseract
from os.path import exists, join, isfile, isdir
from os import listdir
from shutil import copyfile
import os

traineddata_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi'
tessdata_folder='/usr/share/tesseract-ocr/4.00/tessdata'
os.putenv("TESSDATA_PREFIX", tessdata_folder)
if not exists(join(traineddata_folder,'LCDDot_FT_500.traineddata')):
  !wget https://github.com/ameera3/OCR_Expiration_Date/blob/master/OCR_Ensemble/LCDDot_FT_500.traineddata -P "$traineddata_folder"
if not exists(join(traineddata_folder,'dotslayer.traineddata')):
  !wget https://github.com/Shreeshrii/tessdata_shreetest/blob/master/dotslayer.traineddata -P "$traineddata_folder"
# Trovare per quale motivo funziona installare la lingua italiana per tesseract mentre non funziona, neanche con l'italiano scaricato dalla pagina github ufficiale dei file di tesseract trainati, portarli nella cartella. Probabile ci sia un comando per fare il refresh di tesseract
if not exists(join(traineddata_folder,'ita.traineddata')):
  !wget https://github.com/tesseract-ocr/tessdata/blob/master/ita.traineddata -P "$traineddata_folder"

#copyfile(join(traineddata_folder,'LCDDot_FT_500.traineddata'), join(tessdata_folder,'LCDDot_FT_500.traineddata'))
#copyfile(join(traineddata_folder,'dotslayer.traineddata'), join(tessdata_folder,'dotslayer.traineddata'))
#copyfile(join(traineddata_folder,'ita.traineddata'), join(tessdata_folder,'ita.traineddata'))

# Sintassi generale per usare il comando con le opzioni di configurazione per tesseract
#target = pytesseract.image_to_string(image, lang='eng', boxes=False, config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')
# Da prelevare l'immagine giusta o la cartella con le immagini di tutte le boundibg box ritagliate
if 0==1: # Da fare al pc, qui da solo errori usando altri file trainati
  detected_folder='/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_2'
  folders=[d for d in listdir(detected_folder) if isdir(join(detected_folder,d))]
  for folder in ['buoni_refiner_3','scarti_refiner_3']: #folders:
    images=[d for d in listdir(join(detected_folder,folder)) if isdir(join(detected_folder,folder,d))]
    images.sort()
    for image in images:
      files=[f for f in listdir(join(detected_folder,folder,image)) if isfile(join(detected_folder,folder,image,f))]
      LCDDot_FT_500=[]
      dotslayer=[]
      ita=[]
      eng=[]
      for file in files:
        # In caso si utilizzi il metodo 2: psm=8 per indicare a tesseract che dovra' rilevare una singola parola, oem=3 di default e nessuna whitelist per ora
        # Senza usare il file trainato per il font a matrice di punti
        # Ha bisogno dell'utilizzo della funzione link dots per ingrassare i punti e cercare di rendere il font normale
        img=join(detected_folder,folder,image,file)
        print(img+'({})'.format(labels[file]))
        # Suppongo si possano togliere caratteri tipo la O o lo 0, la I o L'1, la A con il 4, Z con 2
        # (per evitare casi ambigui suppongo si possano stampare solo determinate combinazioni con meno caratteri, come nelle targhe... Poi si puo' modificare in ogni caso)
        LCDDot_FT_500.append(pytesseract.image_to_string(img, lang='LCDDot_FT_500', config="-c tessedit_char_whitelist=123456789ABCDEFGHJKLMNOPQRSTUVWXY.-:").replace('\n','#').replace(' ',''))
        dotslayer.append(pytesseract.image_to_string(img, lang='dotslayer', config="-c tessedit_char_whitelist=123456789ABCDEFGHJKLMNOPQRSTUVWXY.-:").replace('\n','#').replace(' ',''))
        ita.append(pytesseract.image_to_string(img, lang='ita', config="-c tessedit_char_whitelist=123456789BCDEFGHJKLMNOPQRSTUVWXY.-:").replace('\n','#').replace(' ',''))
        eng.append(pytesseract.image_to_string(img, lang='eng', config="-c tessedit_char_whitelist=123456789BCDEFGHJKLMNOPQRSTUVWXY.-:").replace('\n','#').replace(' ',''))
        print("\tTesseract result (LCDDot_FT_500.traineddata):",LCDDot_FT_500)
        print("\tTesseract result (dotslayer.traineddata):",dotslayer)
        print("\tTesseract result (ita.traineddata):",ita)
        print("\tTesseract result (eng.traineddata):",eng)

/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_2/buoni_3/Img0/0.jpg
LIOTTTICLI###
/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_2/buoni_3/Img0/0_flipped.jpg
"TRILLO###
/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_2/buoni_3/Img0/1.jpg
i ZOLCOH###
/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_2/buoni_3/Img0/1_flipped.jpg
UNO###
/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_2/buoni_3/Img0/2.jpg

/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_2/buoni_3/Img0/2_flipped.jpg

/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_2/buoni_3/Img0/3.jpg

/content/drive/Shareddrives/Tesi Martignetti-Papallazi/Pictures SALF/Verifica Lotto/metodo_2/buoni_3/Img0/3_flipped.jpg

/content/drive/S

# Analisi dei risultati


## Con le liste
<b>TODO</b>: integrare la lettura di tesseract

In [ ]:
# correct content of the images
labels = [[0, 'Img0.jpg', 'L20L0M', '12-2022'],
          [0, 'Img0_flipped.jpg', 'L20L0M', '12-2022'],
          [0, 'Img1.jpg', 'L20L0M', '12-2022'],
          [0, 'Img1_flipped.jpg', 'L20L0M', '12-2022'],
          [1, 'Img2.jpg', 'L20K77', '11-2022'],
          [1, 'Img2_flipped.jpg', 'L20K77', '11-2022'],
          [1, 'Img3.jpg', 'L20K77', '11-2022'],
          [1, 'Img3_flipped.jpg', 'L20K77', '11-2022'],
          [2, 'Img4.jpg', 'L20K6X', '11-2023'],
          [2, 'Img4_flipped.jpg', 'L20K6X', '11-2023'],
          [2, 'Img5.jpg', 'L20K6X', '11-2023'],
          [2, 'Img5_flipped.jpg', 'L20K6X', '11-2023'],
          [2, 'Img6.jpg', 'L20K6X', '11-2023'],
          [2, 'Img6_flipped.jpg', 'L20K6X', '11-2023'],
          [2, 'Img7.jpg', 'L20K6X', '11-2023'],
          [2, 'Img7_flipped.jpg', 'L20K6X', '11-2023'],
          [3, 'Img8.jpg', 'L20L09', '12-2022'],
          [3, 'Img8_flipped.jpg', 'L20L09', '12-2022'],
          [3, 'Img9.jpg', 'L20L09', '12-2022'],
          [3, 'Img9_flipped.jpg', 'L20L09', '12-2022'],]
# reading of tesseract
tesseract_read = range(len(labels))
# 
right = 0
wrong = 0
for i in tesseract_read:
  if tesseract_read[i] == labels[i]:
    right = right + 1
  else: wrong = wrong + 1

print('right: ' + str(right))
print('wrong: ' + str(wrong))
print(str(right/wrong*100) + '%')

right: 0
wrong: 20
0.0%
